## Making the trianable ansatz

In [14]:
import numpy as np

In [15]:
def nestedKronecker(args): # use "*args" to access an array of inputs
    assert len(args) >= 2
    temp = args[0]
    for arg in args[1:]:
        temp = np.kron(temp, arg)
    return temp

basis = {0: [1,0], 1: [0,1], '0': [1,0], '1': [0,1]}

basisVector = lambda binstr : nestedKronecker([basis[x] for x in binstr])

# common states
zero, one = basis['0'], basis['1']
tplus = basisVector('11')
tminus = basisVector('00')
tzero = (1/np.sqrt(2))*(basisVector('01') + basisVector('10'))
singlet = np.sqrt(1/2)*(basisVector('01') - basisVector('10'))


# ------------------------ FOR STATE 1 ------------------------

state1 = np.kron(np.kron(singlet, singlet), singlet)

# ------------------------ FOR STATE 2 ------------------------

largePyramid = np.sqrt(1/3)*(np.kron(tplus,tminus)+np.kron(tminus,tplus)-np.kron(tzero,tzero))
state2 = np.kron(singlet,largePyramid)

# ------------------------ FOR STATE 3 ------------------------

state3 = np.kron(largePyramid,singlet)

# ------------------------ FOR STATE 4 ------------------------

# for psi0 and psi1 we are combining j1=1 and j2=1/2 (this is combinind the first peak and trough)
# J = 1/2, M = -1/2
psi0 = np.sqrt(1/3)*np.kron(tzero, zero) - np.sqrt(2/3)*np.kron(tminus, one)
# J = 1/2, M = +1/2
psi1 = np.sqrt(2/3)*np.kron(tplus, zero) - np.sqrt(1/3)*np.kron(tzero, one)


# for phiminus, phizero, phiplus, we are are combining j1=1/2 and j2=1/2
# J = 1, M = -1
phiminus = np.kron(psi0,zero)
# J = 1, M = 0
phizero = np.sqrt(1/2)*(np.kron(psi1,zero) + np.kron(psi0,one))
# J = 1, M = +1
phiplus = np.kron(psi1,one)

# J=0,M=0 and j1=1,j2=1
state4 = np.sqrt(1/3)*(np.kron(phiplus, tminus) - np.kron(phizero, tzero) + np.kron(phiminus, tplus))

# ------------------------ FOR STATE 5 ------------------------

eta_minus3 = np.kron(tminus, basis['0'])
eta_minus1 = np.sqrt(2/3)*np.kron(tzero,zero) + np.sqrt(1/3)*np.kron(tminus,one)
eta_plus1 = np.sqrt(1/3)*np.kron(tplus,zero) + np.sqrt(2/3)*np.kron(tzero, one)
eta_plus3 = np.kron(tplus,one)

gamma_minus = np.sqrt(1/4)*np.kron(eta_minus1, zero) - np.sqrt(3/4)*np.kron(eta_minus3, one)
gamma_zero = np.sqrt(1/2)*np.kron(eta_plus1, zero) - np.sqrt(1/2)*np.kron(eta_minus1,one)
gamma_plus = np.sqrt(3/4)*np.kron(eta_plus3, zero) - np.sqrt(1/4)*np.kron(eta_plus1, one)

state5 = np.sqrt(1/3)*(np.kron(gamma_plus,tminus) - np.kron(gamma_zero, tzero) - np.kron(gamma_minus, tplus))

inputStates = np.array([state1, state2, state3, state4])
expectedStates = np.array([state1, state2, state4, state3])

In [24]:
import numpy as np
from scipy.optimize import minimize
import pennylane as qml

I = np.eye(2)
n_qubits = 6
size_of_vec = 2**n_qubits
num_layers = 7
inputStates = np.array([state1, state2, state3, state4])
expectedStates = np.array([state1, state2, state4, state3])

# Trying ChatPGT

In [22]:
import numpy as np
from scipy.optimize import minimize
import pennylane as qml


Udot = lambda s1, U, s2 : np.dot(np.conjugate(np.transpose(s1)),np.matmul(U,s2))

def nestedKron(*args): # use "*args" to access an array of inputs
    assert len(args) >= 2
    temp = args[0]
    for arg in args[1:]:
        temp = np.kron(temp, arg)
    return temp

def get_random_weights(num_layers):
    return 2 * np.pi * np.random.random(size=(num_layers, 5)) - np.pi

def U_ex(p):
    from scipy.linalg import expm
    X = [[0,1],[1,0]]
    Y = np.array([[0,-1j],[1j,0]], dtype=np.complex128)
    Z = [[1,0],[0,-1]]

    H_ex = (1/4)*(np.kron(X,X) + np.kron(Y,Y) + np.kron(Z,Z))
    # print(f'H_ex.type = {type(H_ex)}')
    U_exchange = expm(-1j*p*H_ex) # p is now -pi to pi
    return np.array(U_exchange)

def single_layer_U(layer_weights):
    """Trainable circuit block."""
    firstPart = nestedKron(U_ex(layer_weights[0]), U_ex(layer_weights[1]), U_ex(layer_weights[2]))
    secondPart = nestedKron(I, U_ex(layer_weights[3]), U_ex(layer_weights[4]), I)
    return np.matmul(secondPart, firstPart)

def get_matrix(weights):
    totalMatrix = np.eye(size_of_vec)
    for layer_weights in weights:
        mat = single_layer_U(layer_weights)
        totalMatrix = np.matmul(totalMatrix, mat)
    return totalMatrix

def square_loss(y_true, y_pred):
    loss = 0
    for i in range(len(expectedStates)):
        # c = np.dot(np.conjugate(expectedStates[i]), predictedStates[i])
        # c_2 = self.amp_sqrd(c)
        fidelity = qml.math.fidelity_statevector(y_true[i], y_pred[i])
        loss += (1 - fidelity) ** 2
    loss /= len(expectedStates)
    return 0.5*loss

def f_cnot_loss(y_true, y_pred):
    loss = 0
    for i in range(len(expectedStates)):
        fidelity = qml.math.fidelity_statevector(y_true[i], y_pred[i])
        loss += fidelity
    return np.sqrt(1 - (1/4)*abs(loss))

# Define the correct operations you want the matrix to perform on basis vectors
def target_operations(parameters, inputStates):
    # Reshape the parameters into the matrix form
    parameters = np.reshape(parameters, (num_layers, 5))
    matrix = get_matrix(parameters)

    # Perform matrix multiplication with basis vectors
    results = []
    for i in range(len(inputStates)):
        results.append(np.matmul(matrix, inputStates[i]))

    # Define the target operations you want (modify this based on your specific task)
    target_result = np.array(expectedStates)

    # Calculate the loss as the difference between the obtained result and the target result
    # loss = square_loss(target_result, results)
    loss = f_cnot_loss(target_result, results)
    return loss

# Example: Set the number of basis vectors and their dimensionality
num_vectors = 4
vector_dimension = size_of_vec

# Generate random basis vectors and target result
basis_vectors = np.array(inputStates)
target_result = np.array(expectedStates)

# Flatten the matrix parameters for optimization
initial_parameters = np.ndarray.flatten(get_random_weights(num_layers))

# Use scipy's minimize function to optimize the parameters
result = minimize(target_operations, initial_parameters, args=(basis_vectors,), method='L-BFGS-B')

# Reshape the optimized parameters back into the matrix form
optimized_matrix = get_matrix(result.x.reshape((num_layers, 5)))

print("Optimized Matrix:")
print(optimized_matrix)

predStates = [np.matmul(optimized_matrix, mat) for mat in inputStates]
print(f"f_cnot_loss = {f_cnot_loss(expectedStates, predStates)}")
print(f"square_loss = {square_loss(expectedStates, predStates)}")

TypeError: minimize() got an unexpected keyword argument 'disp'

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

def f_cnot_loss(y_true, y_pred):
    loss = 0
    for i in range(np.size(y_true)//(2**n_qubits)):
        fidelity = qml.math.fidelity_statevector(y_true[i], y_pred[i])
        loss += fidelity
    return np.sqrt(1 - (1/4)*abs(loss))


# Load and preprocess the MNIST dataset
x_train, y_train = inputStates, expectedStates

param_size = num_layers*5
parameters = np.array(get_random_weights(num_layers))
params = np.ndarray.flatten(parameters).astype('float32')




# Build the neural network model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(param_size*2, activation='relu'),
    layers.Dense(param_size, activation='tanh'),
])

# Compile the model
model.compile(optimizer='adam',
              loss=f_cnot_loss,
              metrics=['accuracy'])

# Train the model
model.fit(train_images, train_labels, epochs=5, batch_size=64)

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f'Test accuracy: {test_acc}')


In [41]:
import tensorflow as tf
import numpy as np

def f_cnot_loss(y_true, y_pred):
    loss = 0
    for i in range(np.size(y_true)//(2**n_qubits)):
        fidelity = qml.math.fidelity_statevector(y_true[i], y_pred[i])
        loss += fidelity
    return np.sqrt(1 - (1/4)*abs(loss))


# Define the correct operations you want the matrix to perform on basis vectors
def target_operations(parameters, inputVectors):
    # Perform matrix multiplication with basis vectors
    # Reshape the parameters into the matrix form
    params = np.reshape(parameters, (num_layers, 5))
    matrix = tf.constant(get_matrix(params), dtype=tf.complex128)
    # results = tf.matmul(matrix, inputVectors, transpose_b=True)
    results = []
    for i in range(np.size(inputVectors)//(2**n_qubits)):
        results.append(np.matmul(matrix, inputVectors[i]))
    results = tf.constant(results, dtype=tf.complex128)

    # Define the target operations you want (modify this based on your specific task)
    target_results = tf.constant(expectedStates, dtype=tf.complex128)

    # Calculate the loss as the mean squared error between the obtained result and the target result
    loss = f_cnot_loss(target_results, results)
    return loss

# Example: Set the number of basis vectors and their dimensionality
num_vectors = 4
vector_dimension = 2**n_qubits

# Generate random basis vectors and target result
inputVectors = tf.constant(inputStates, dtype=tf.complex128)

# Define the matrix as a TensorFlow Variable
parameters = tf.Variable(np.ndarray.flatten(get_random_weights(num_layers)), dtype=tf.float32)

# Use an optimizer to minimize the loss
optimizer = tf.optimizers.Adam(learning_rate=0.01)

# Training loop
epochs = 1000
for epoch in range(epochs):
    with tf.GradientTape() as tape:
        loss = tf.constant(target_operations(parameters, inputVectors))

    gradients = tape.gradient(loss, [parameters])
    optimizer.apply_gradients(zip(gradients, [parameters]))

    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss.numpy()}")

# Get the optimized matrix
parameters = np.reshape(parameters, (num_layers, 5))
matrix = get_matrix(parameters)
optimized_matrix = matrix.numpy()

print("Optimized Matrix:")
print(optimized_matrix)

ValueError: No gradients provided for any variable: (['Variable:0'],). Provided `grads_and_vars` is ((None, <tf.Variable 'Variable:0' shape=(35,) dtype=float32, numpy=
array([ 2.5269053 ,  0.5212329 ,  0.9739547 ,  2.5969772 ,  0.6503308 ,
        2.1297295 , -1.931953  ,  0.7041187 , -1.3276408 , -2.2411902 ,
        1.2720861 ,  1.5847887 ,  3.0150158 ,  1.2192807 , -1.7325749 ,
       -0.82495993, -1.756478  , -1.1077476 , -2.713036  ,  1.8335059 ,
        1.614816  ,  2.2740006 ,  2.2506728 , -1.1599189 , -2.0895307 ,
        3.0720863 , -0.31033638, -0.30215982, -2.318698  , -3.134845  ,
       -1.3226986 , -2.8566093 ,  1.5320274 ,  0.39243674,  0.13296835],
      dtype=float32)>),).

array([[1, 2, 3, 4]])

In [129]:
import numpy as np
import pennylane as qml

class MatrixOperations():
    def __init__(self, expectedStates, inputStates, weights, num_layers):
        self.expectedStates = expectedStates
        self.inputStates = inputStates
        self.weights = weights
        self.num_layers = num_layers
        self.Udot = lambda s1, U, s2 : np.dot(np.conjugate(np.transpose(s1)),np.matmul(U,s2))

    def nestedKron(self, *args): # use "*args" to access an array of inputs
        assert len(args) >= 2
        temp = args[0]
        for arg in args[1:]:
            temp = np.kron(temp, arg)
        return temp

    def get_random_weights(self):
        return 2 * np.random.random(size=(self.num_layers, 5)) - 1

    def U_ex(self, p):
        from scipy.linalg import expm
        X = [[0,1],[1,0]]
        Y = np.array([[0,-1j],[1j,0]], dtype=np.complex128)
        Z = [[1,0],[0,-1]]

        H_ex = (1/4)*(np.kron(X,X) + np.kron(Y,Y) + np.kron(Z,Z))
        # print(f'H_ex.type = {type(H_ex)}')
        U_exchange = expm(-1j*np.pi*p*H_ex) # p is -1 to 1
        return np.array(U_exchange)

    def get_predictions(self, weights):
        matOp = self.get_total_matrix(weights)
        results = []
        for i in range(len(self.inputStates)):
            results.append(np.matmul(matOp, self.inputStates[i]))
        return np.array(results)
    
    def count_weight_vals_that_are_zero(self, weights, movement):
        constraint = lambda val : val >= -movement*1.5 and val <= movement*1.5

        count = 0
        for layer in weights:
            for block_val in layer:
                count += 1 if constraint(block_val) else 0
        return count


    def single_layer_U(self,layer_weights):
        """Trainable circuit block."""
        firstPart = self.nestedKron(self.U_ex(layer_weights[0]), self.U_ex(layer_weights[1]), self.U_ex(layer_weights[2]))
        secondPart = self.nestedKron(I, self.U_ex(layer_weights[3]), self.U_ex(layer_weights[4]), I)
        return np.matmul(secondPart, firstPart)

    def get_total_matrix(self,weights):
        totalMatrix = np.eye(size_of_vec)
        for layer_weights in weights:
            mat = self.single_layer_U(layer_weights)
            totalMatrix = np.matmul(totalMatrix, mat)
        return totalMatrix

    def f_cnot_loss(self,y_true, y_pred):
        loss = 0
        for i in range(len(expectedStates)):
            fidelity = qml.math.fidelity_statevector(y_true[i], y_pred[i])
            loss += fidelity
        return np.sqrt(1 - (1/4)*abs(loss))
    
    def cost_fn(self, weights):
        preds = self.get_predictions(weights)
        loss = self.f_cnot_loss(self.expectedStates, preds)
        return loss

    # Define the correct operations you want the matrix to perform on basis vectors
    def target_operations(self,parameters):
        # Reshape the parameters into the matrix form
        parameters = np.reshape(parameters, (self.num_layers, 5))
        matrix = self.get_total_matrix(parameters)

        # Perform matrix multiplication with basis vectors
        results = self.get_predictions(parameters)

        # Define the target operations you want (modify this based on your specific task)
        target_result = np.array(self.expectedStates)

        # Calculate the loss as the difference between the obtained result and the target result
        # loss = square_loss(target_result, results)
        loss = f_cnot_loss(target_result, results)
        return loss

In [36]:
import gym
num_layers=7
action_space = gym.spaces.Box(low=-1, high=1, shape=(num_layers,5,))

state = 2*np.random.random(size=(7,5))-1
state += action_space.sample()
size = lambda boxSpace : boxSpace.shape[0] * boxSpace.shape[1]
size(action_space), np.array([1,2]).size
import math
def computeRewardAndUpdateState():
    cond = lambda loss : True if loss<0.1 else False
    getBoost = lambda loss : int(math.ceil(-math.log2(loss**3))*15) if cond(loss) else 0
    getExtraZeroGateBoost = lambda loss, zero_counts : int(math.ceil(-math.log2((1/zero_counts)**10)))*10 if zero_counts>=1 and cond(loss) else zero_counts*5
    loss = 1e-17
    zero_counts = 0
    totalReward = (1-loss)*500 + getBoost(loss) + getExtraZeroGateBoost(loss, zero_counts)

    return totalReward

computeRewardAndUpdateState()

3050.0

In [ ]:
import tensorflow as tf
import numpy as np


from reinforcement_learning.Env import QuantumEnvironment

# Define the reinforcement learning model using a simple neural network
def build_model(input_dim, output_dim):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(64, activation='relu', input_shape=(input_dim,)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(output_dim, activation='tanh')  # Use tanh to ensure output in the range [-1, 1]
    ])
    return model

# Define the Deep Deterministic Policy Gradient (DDPG) algorithm
class DDPGAgent:
    def __init__(self, state_dim, action_dim):
        self.actor = build_model(state_dim, action_dim) # this only gives best action to take based on state as input
        self.critic = build_model(state_dim + action_dim, 1) # this takes both state and action from actor to evaluate how good it is
        self.actor_optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
        self.critic_optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
# guassian noise needed for explore-explot dilemma -----------------------------

    def get_action(self, state):
        return self.actor(state.reshape(1, -1)).numpy().flatten()

    def train(self, states, actions, rewards, next_states):
        # Define the actor and critic training steps using the DDPG algorithm
        pass
# squared bellman error loss -----------------------
# Hyperparameters
num_parameters = 10
state_dim = 4  # Adjust based on your state representation
action_dim = num_parameters

# Initialize quantum environment and DDPG agent
env = QuantumEnvironment(num_parameters)
agent = DDPGAgent(state_dim, action_dim)

# Training loop
for episode in range(num_episodes):
    state = env.reset()
    total_reward = 0

    for _ in range(num_steps):
        action = agent.get_action(state)
        next_state, reward = env.step(action)
        agent.train(state, action, reward, next_state)

        state = next_state
        total_reward += reward

    print(f"Episode: {episode + 1}, Total Reward: {total_reward}")


In [45]:
import gym
env = gym.Env()
env.action_space()

AttributeError: 'Env' object has no attribute 'action_space'

In [17]:
p1 = np.arccos(-1/np.sqrt(3))/np.pi
p2 = np.arcsin(1/3)/np.pi


def Id_n(n):
    assert n >= 0
    if n==0:
        return 1
    temp = I
    for i in range(n-1):
        temp = np.kron(temp, I)
    return temp


bounds = [[3,4],[2,5],[3,4],[2,5],[1,4],[2,5],[1,4],[2,5],[1,4],[2,5],[3,4],[2,5],[3,4]]
operators = [U_ex(p1),
np.kron(U_ex(1/2),U_ex(p2)),
U_ex(1),
np.kron(U_ex(-1/2),U_ex(-1/2)),
np.kron(U_ex(1),U_ex(-1/2)),
np.kron(U_ex(-1/2),U_ex(1)),
np.kron(U_ex(-1/2),U_ex(1/2)),
np.kron(U_ex(-1/2),U_ex(1)),
np.kron(U_ex(1),U_ex(-1/2)),
np.kron(U_ex(-1/2),U_ex(-1/2)),
U_ex(1),
np.kron(U_ex(1/2),U_ex(1-p2)),
U_ex(-p1)]


newOps = []
for i, (start, end) in enumerate(bounds):
    temp = nestedKron(Id_n(start),operators[i], Id_n(5-end))
    newOps.append(temp.copy())

totalOperator = np.eye(2**6)
for op in newOps:
    totalOperator = np.matmul(op,totalOperator)

U_cnot = totalOperator.copy()


predStates = [np.matmul(U_cnot, mat) for mat in inputStates]
print(f"square loss = {square_loss(expectedStates, predStates)}")

square loss = 0.14172274545673982


In [21]:
from QNN import Ansatz
ansatz = Ansatz(num_layers=num_layers)
ansatz.weights = result.x.reshape((num_layers, 5))
predStates = ansatz.get_predictions(ansatz.weights, inputStates)
loss = ansatz.square_loss(expectedStates, predStates)

print(f'loss from the pennylane qml model is = {loss}')


loss from the pennylane qml model is = 0.3678316991400055
